## Notebook to determine important carriers for the STST and EXP networks

In [1]:
# imports
import pypsa
import pandas as pd

from utils import nodal_balance
from utils import get_condense_sum, carrier_renaming, carrier_renaming_reverse
from utils import resistive_heater, gas_boiler, heat_pump, water_tanks_charger, water_tanks_discharger, solar_thermal

# Notebook Definitions
c1_groups = [resistive_heater, gas_boiler, heat_pump, water_tanks_charger, water_tanks_discharger, solar_thermal]
c1_groups_name = ["resistive heater", "gas boiler", "heat pump", "water tanks charger", "water tanks discharger", "solar thermal"]

# stst and exp dataframes
df_stst_ons = pd.read_pickle("../data/processed/df_stst_ons.pkl")
df_stst_off = pd.read_pickle("../data/processed/df_stst_off.pkl")
df_exp_ons = pd.read_pickle("../data/processed/df_exp_ons.pkl")
df_exp_off = pd.read_pickle("../data/processed/df_exp_off.pkl")

In [2]:
# Network imports
stst = pypsa.Network("../data/raw/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2030.nc")
exp = pypsa.Network("../data/raw/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2030.nc")

Importing PyPSA from older version of PyPSA than current version.
Please read the release notes at https://pypsa.readthedocs.io/en/latest/release_notes.html
carefully to prepare your network for import.
Currently used PyPSA version [0, 20, 1], imported network file PyPSA version [0, 18, 1].

INFO:pypsa.io:Imported network elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2030.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
Importing PyPSA from older version of PyPSA than current version.
Please read the release notes at https://pypsa.readthedocs.io/en/latest/release_notes.html
carefully to prepare your network for import.
Currently used PyPSA version [0, 20, 1], imported network file PyPSA version [0, 18, 1].

INFO:pypsa.io:Imported network elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2030.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


In [3]:
print(stst)

PyPSA Network
Components:
 - Bus: 3086
 - Carrier: 30
 - Generator: 1707
 - GlobalConstraint: 3
 - Line: 323
 - Link: 8375
 - Load: 2542
 - StorageUnit: 170
 - Store: 1635
Snapshots: 2920


In [4]:
print(exp)

PyPSA Network
Components:
 - Bus: 3086
 - Carrier: 30
 - Generator: 1707
 - GlobalConstraint: 2
 - Line: 323
 - Link: 9192
 - Load: 2542
 - StorageUnit: 170
 - Store: 1635
Snapshots: 2920


### Carrier differences

In [5]:
# same carriers for generators
set(exp.generators.carrier.unique().tolist()).difference(stst.generators.carrier.unique().tolist())
stst.generators.carrier.unique().tolist()

['offwind-ac',
 'onwind',
 'solar',
 'ror',
 'offwind-dc',
 'gas',
 'residential rural solar thermal',
 'services rural solar thermal',
 'residential urban decentral solar thermal',
 'services urban decentral solar thermal',
 'urban central solar thermal',
 'oil',
 'solar rooftop']

In [6]:
# EXP has two additional links: 'H2 pipeline', 'H2 pipeline retrofitted'
set(exp.links.carrier.unique().tolist()).difference(stst.links.carrier.unique().tolist())
exp.links.carrier.unique().tolist()

['DC',
 'OCGT',
 'H2 Electrolysis',
 'H2 Fuel Cell',
 'H2 pipeline retrofitted',
 'H2 pipeline',
 'battery charger',
 'battery discharger',
 'Sabatier',
 'SMR CC',
 'SMR',
 'BEV charger',
 'V2G',
 'residential rural ground heat pump',
 'residential rural water tanks charger',
 'residential rural water tanks discharger',
 'residential rural resistive heater',
 'residential rural gas boiler',
 'services rural ground heat pump',
 'services rural water tanks charger',
 'services rural water tanks discharger',
 'services rural resistive heater',
 'services rural gas boiler',
 'residential urban decentral air heat pump',
 'residential urban decentral water tanks charger',
 'residential urban decentral water tanks discharger',
 'residential urban decentral resistive heater',
 'residential urban decentral gas boiler',
 'services urban decentral air heat pump',
 'services urban decentral water tanks charger',
 'services urban decentral water tanks discharger',
 'services urban decentral resisti

In [7]:
# same carriers for storage units
set(stst.storage_units.carrier.unique().tolist()).difference(exp.storage_units.carrier.unique().tolist())
stst.storage_units.carrier.unique().tolist()

['hydro', 'PHS']

In [8]:
# overall number of carriers:
# STST: 68
# EXP: 70
len(stst.storage_units.carrier.unique().tolist() + stst.links.carrier.unique().tolist() + stst.generators.carrier.unique().tolist())

68

### Relevant carriers per technology
- classify in electricity (gen / con), hydrogen (gen, con) and heat (gen, con)
- determine by share of generation / consumption

In [9]:
# electricity

# overall electricity generation in TWh per technology
carrier = ["AC", "low voltage"]
nb = nodal_balance(stst, carrier=carrier, time="2013", aggregate=['component', 'bus'], energy=True)
# convert from MWh to GWh
nb = nb.unstack(level=[1]) / 1000
# condense condense_groups
nb = get_condense_sum(nb, c1_groups, c1_groups_name)
# rename unhandy column names
nb.rename(columns=carrier_renaming, inplace=True)
# calc sum and convert to TWh
(nb[nb > 0].sum() / 1000).sort_values()

carrier
solar thermal                 0.000000
BEV charger                   0.000000
DAC                           0.000000
water tanks charger           0.000000
H2 Electrolysis               0.000000
heat pump                     0.000000
gas boiler                    0.000000
resistive heater              0.000000
industry electricity          0.000000
agriculture electricity       0.000000
battery charger               0.000000
water tanks discharger        0.000000
electricity                   0.000000
home battery charger          0.000000
AC                            0.000009
DC                            0.000029
gas CHP CC                    0.007028
home battery discharger       0.130558
H2 Fuel Cell                  1.409125
OCGT                          3.031344
PHS                          46.621278
V2G                          99.732769
biomass CHP CC              105.337329
ror                         138.641227
gas CHP                     139.465576
offwind-ac       

In [10]:
pos_gen = (nb[nb > 0].sum() / 1000).sort_values(ascending=False)[(nb[nb > 0].sum() / 1000).sort_values(ascending=False) > 0.001]
pos_gen.index = [carrier_renaming_reverse.get(n, n) for n in pos_gen.index]
pos_gen.sort_values(ascending=False)

onwind                                4556.001892
solar                                 3255.956798
offwind-dc                             609.327531
battery discharger                     388.014856
hydro                                  348.187357
solar rooftop                          328.158738
offwind-ac                             182.048413
urban central gas CHP                  139.465576
ror                                    138.641227
urban central solid biomass CHP CC     105.337329
V2G                                     99.732769
PHS                                     46.621278
OCGT                                     3.031344
H2 Fuel Cell                             1.409125
home battery discharger                  0.130558
urban central gas CHP CC                 0.007028
dtype: float64

In [11]:
(df_stst_ons[[c + "_gen" for c in pos_gen.index]].sum() / 1000) / (df_stst_ons[[c + "_gen" for c in pos_gen.index]].sum() / 1000).sum()

onwind_gen                                0.446041
solar_gen                                 0.318764
offwind-dc_gen                            0.059655
battery discharger_gen                    0.037989
hydro_gen                                 0.034089
solar rooftop_gen                         0.032128
offwind-ac_gen                            0.017824
urban central gas CHP_gen                 0.013654
ror_gen                                   0.013573
urban central solid biomass CHP CC_gen    0.010313
V2G_gen                                   0.009767
PHS_gen                                   0.005749
OCGT_gen                                  0.000297
H2 Fuel Cell_gen                          0.000139
home battery discharger_gen               0.000017
urban central gas CHP CC_gen              0.000002
dtype: float64

In [12]:
# electricity

# overall electricity generation in TWh per technology
carrier = ["AC", "low voltage"]
nb = nodal_balance(exp, carrier=carrier, time="2013", aggregate=['component', 'bus'], energy=True)
# convert from MWh to GWh
nb = nb.unstack(level=[1]) / 1000
# condense condense_groups
nb = get_condense_sum(nb, c1_groups, c1_groups_name)
# rename unhandy column names
nb.rename(columns=carrier_renaming, inplace=True)
# calc sum and convert to TWh
(nb[nb > 0].sum() / 1000).sort_values()

carrier
industry electricity       0.000000e+00
water tanks charger        0.000000e+00
heat pump                  0.000000e+00
gas boiler                 0.000000e+00
resistive heater           0.000000e+00
water tanks discharger     0.000000e+00
home battery charger       0.000000e+00
electricity                0.000000e+00
solar thermal              0.000000e+00
agriculture electricity    0.000000e+00
BEV charger                0.000000e+00
H2 Electrolysis            0.000000e+00
battery charger            0.000000e+00
AC                         5.688746e-07
DC                         2.994737e-05
home battery discharger    7.775209e-03
OCGT                       2.957557e+00
biomass CHP                2.709590e+01
PHS                        4.170369e+01
gas CHP                    4.740043e+01
V2G                        6.455862e+01
biomass CHP CC             8.104071e+01
battery discharger         1.085450e+02
ror                        1.393801e+02
offwind-ac                 1.441

In [13]:
pos_gen = (nb[nb < 0].sum() / 1000).sort_values(ascending=False)[(nb[nb < 0].sum() / 1000).sort_values(ascending=False) < -0.001]
pos_gen.index = [carrier_renaming_reverse.get(n, n) for n in pos_gen.index]
pos_gen.sort_values(ascending=True)

H2 Electrolysis                  -3558.103289
electricity                      -1720.767554
industry electricity             -1702.520000
BEV charger                       -946.278666
heat pump                         -787.444014
urban central resistive heater    -239.795041
battery charger                   -113.068737
PHS                                -59.385843
agriculture electricity            -44.861105
home battery charger                -0.015978
dtype: float64

In [14]:
(df_stst_ons[[c + "_gen" for c in carriers_el_gen]].sum() / 1000).sort_values(ascending=False)

NameError: name 'carriers_el_gen' is not defined

In [ ]:
(df_exp_ons[[c + "_gen" for c in carriers_el_gen]].sum() / 1000).sort_values(ascending=False)

In [ ]:
# investigate in carriers with most generation for both networks
# without grid ("electricity distribution grid", "DC", "H2 pipeline", "H2 pipeline retrofitted")
# storage technologies:

carriers_el_gen = [
    "electricity distribution grid",
    "onwind",
    "solar",
    "BEV charger",
    "offwind-dc",
    "ror",
    "hydro",
    "solar rooftop",
    "offwind-ac",
    "battery charger",
    "battery discharger",
    "urban central solid biomass CHP CC",
    "V2G",
    "PHS",
    "urban central gas CHP",
    "urban central solid biomass CHP",
    "OCGT",
    "home battery charger",
    "home battery discharger",
    "H2 Fuel Cell",
    "urban central gas CHP CC",
    "DC"
]

carriers_el_con = [
    "PHS",
    "electricity distribution grid",
    "H2 Electrolysis",
    "BEV charger",
    "urban central air heat pump",
    "residential urban decentral air heat pump",
    "residential rural ground heat pump",
    "services rural ground heat pump",
    "services urban decentral air heat pump",
    "urban central resistive heater",
    "battery charger",
    "battery discharger",
    "V2G",
    "residential urban decentral resistive heater",
    "residential rural resistive heater",
    "services urban decentral resistive heater",
    "services rural resistive heater",
    "home battery charger",
    "home battery discharger",
    "DC",
    "DAC",
]

carriers_h2_gen = [
    "H2 Electrolysis",
    "SMR CC",
    "SMR"
]

carriers_h2_con = [
    "H2 liquefaction",
    "H2 Fuel Cell",
    "Sabatier",
    "Fischer-Tropsch"
]

carriers_heat_gen1 = [
    "urban central air heat pump",
    "residential urban decentral air heat pump",
    "residential rural ground heat pump",
    "services rural ground heat pump",
    "services urban decentral air heat pump",
    "urban central gas boiler",
    "urban central resistive heater",
    "residential urban decentral gas boiler",
    "urban central water tanks charger",
    "urban central water tanks discharger",
    "residential urban decentral resistive heater",
    "services urban decentral gas boiler",
    "urban central solar thermal",
    "residential rural resistive heater",
    "services urban decentral resistive heater",
    "residential rural gas boiler",
    "services rural resistive heater",
    "services rural gas boiler",
    "services rural water tanks charger",
    "residential rural water tanks charger",
    "services rural water tanks discharger",
    "residential rural water tanks discharger",
    "services urban decentral water tanks charger",
    "residential urban decentral water tanks charger",
    "services urban decentral water tanks discharger",
    "residential urban decentral water tanks discharger",
    "residential urban decentral solar thermal",
    "services urban decentral solar thermal",
    "residential rural solar thermal",
    "services rural solar thermal"
]

carriers_heat_gen2 = [
    "urban central solid biomass CHP CC",
    "urban central gas CHP",
    "urban central solid biomass CHP",
    "H2 Fuel Cell",
    "urban central gas CHP CC",
    "DAC"
]

carriers_heat_con = [
"urban central water tanks charger",
"urban central water tanks discharger",
"services rural water tanks charger",
"residential rural water tanks charger",
"services rural water tanks discharger",
"residential rural water tanks discharger",
"services urban decentral water tanks charger",
"residential urban decentral water tanks charger",
"services urban decentral water tanks discharger",
"residential urban decentral water tanks discharger",
]

# for gas, biomass, oil, process emission, C02 refer to excel table

In [ ]:
# select carriers which make most sense

# ordered according to (df_stst_ons[[c + "_gen" for c in c_el_gen_s]].sum() + df_exp_ons[[c + "_gen" for c in c_el_gen_s]].sum()).sort_values(ascending=False)
# overall less than 50 TWh: urban central gas CHP CC, fuel cell, home battery discharger, OCGT, urban central solid biomass CHP,
c_el_gen_s = [
    "onwind",
    "solar",
    "offwind-dc",
    "hydro",
    "solar rooftop",
    "battery discharger",
    "offwind-ac",
    "ror",
    "urban central gas CHP",
    "urban central solid biomass CHP CC",
    "V2G",
    "PHS"
]

# use urban central air heat pump as only heat pump (max generation)
# use urban central resistive heater (max generation)
# overall less than 50 TWh: home battery charger
# ordered according to (df_stst_ons[[c + "_con_el" for c in c_el_con_s]].sum() + df_exp_ons[[c + "_con_el" for c in c_el_con_s]].sum()).sort_values(ascending=False)
c_el_con_s = [
    "H2 Electrolysis",
    "BEV charger",
    "battery charger",
    "urban central air heat pump",
    "urban central resistive heater",
    "PHS",
]

carriers_h2_gen = [
    "H2 Electrolysis",
    "SMR CC",
    "SMR"
]

carriers_h2_con = [
    "H2 liquefaction",
    "H2 Fuel Cell"
    "Sabatier",
    "Fischer-Tropsch"
]

# use urban central air heat pump as only heat pump (max generation)
# use urban central resistive heater (max generation)
# use urban central gas boiler (max generation)
# use urban central water tanks charger (max generation)
# use urban central solar thermal (max generation)

carriers_heat_gen1 = [
    "urban central air heat pump"
    "urban central gas boiler"
    "urban central resistive heater"
    "urban central water tanks discharger"
    "urban central solar thermal"
]

carriers_heat_gen2 = [
    "urban central solid biomass CHP CC",
    "urban central gas CHP",
    "urban central solid biomass CHP",
    "H2 Fuel Cell",
    "urban central gas CHP CC",
    "DAC"
]

carriers_heat_con = [
"urban central water tanks charger",
]

# for gas, biomass, oil, process emission, C02 refer to excel table

In [ ]:
n = stst
all_c = n.generators.carrier.unique().tolist() + n.links.carrier.unique().tolist() + n.storage_units.carrier.unique().tolist()

# carriers not in any list:
set(all_c).difference(carriers_el_gen + carriers_el_con + carriers_h2_gen + carriers_h2_con + carriers_heat_gen1 + carriers_heat_con)

In [ ]:
# Check second and third output of links
# what is with DAC?

In [ ]:
n.links[n.links.carrier == "DAC"]

In [ ]:
# exclude technologies with less than 0.5 % of total generation or consumption in both scenarios
stst_car = df_stst_ons[[c + "_gen" for c in carriers_el_gen]].sum()[df_stst_ons[[c + "_gen" for c in carriers_el_gen]].sum() < 51*1e3].index
exp_car   = df_exp_ons[[c + "_gen" for c in carriers_el_gen]].sum()[df_exp_ons[[c + "_gen" for c in carriers_el_gen]].sum() < 46*1e3].index
set(stst_car).intersection(exp_car)

In [ ]:
# Questions Fabian

In [15]:
stst.links[stst.links.carrier == "DAC"]

,bus0,bus1,carrier,geometry,length,p_nom,tags,under_construction,underground,p_min_pu,...,p_nom_opt,charger_ratio,type,build_year,p_nom_min,p_set,p_max_pu,terrain_factor,ramp_limit_up,ramp_limit_down
Link,,,,,,,,,,,,,,,,,,,,,
AL0 0 services urban decentral DAC,co2 atmosphere,co2 stored,DAC,,0.0,0.0,,NaN,NaN,0.0,...,0.009234,NaN,,0,0.0,0.0,1.0,1.0,NaN,NaN
AT0 0 services urban decentral DAC,co2 atmosphere,co2 stored,DAC,,0.0,0.0,,NaN,NaN,0.0,...,0.006710,NaN,,0,0.0,0.0,1.0,1.0,NaN,NaN
AT0 1 services urban decentral DAC,co2 atmosphere,co2 stored,DAC,,0.0,0.0,,NaN,NaN,0.0,...,0.005143,NaN,,0,0.0,0.0,1.0,1.0,NaN,NaN
AT0 2 services urban decentral DAC,co2 atmosphere,co2 stored,DAC,,0.0,0.0,,NaN,NaN,0.0,...,0.006415,NaN,,0,0.0,0.0,1.0,1.0,NaN,NaN
BA0 0 services urban decentral DAC,co2 atmosphere,co2 stored,DAC,,0.0,0.0,,NaN,NaN,0.0,...,0.007397,NaN,,0,0.0,0.0,1.0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SE3 5 urban central DAC,co2 atmosphere,co2 stored,DAC,,0.0,0.0,,NaN,NaN,0.0,...,0.007536,NaN,,0,0.0,0.0,1.0,1.0,NaN,NaN
SE3 6 urban central DAC,co2 atmosphere,co2 stored,DAC,,0.0,0.0,,NaN,NaN,0.0,...,0.008375,NaN,,0,0.0,0.0,1.0,1.0,NaN,NaN
SE3 7 urban central DAC,co2 atmosphere,co2 stored,DAC,,0.0,0.0,,NaN,NaN,0.0,...,0.007742,NaN,,0,0.0,0.0,1.0,1.0,NaN,NaN


In [27]:
stst.links_t.p1.loc[: ,stst.links.carrier == "H2 Fuel Cell"].sum().sum() / 1e6

-0.472020381361872

In [28]:
exp.links_t.p1.loc[: ,exp.links.carrier == "H2 Fuel Cell"].sum().sum() / 1e6

-0.002298661117573675

In [20]:
exp.links_t.p2.loc[: ,exp.links.carrier == "DAC"].sum().sum() / 1e6

0.001333884830489484

In [21]:
stst.links[stst.links.carrier == "urban central gas CHP"]

,bus0,bus1,carrier,geometry,length,p_nom,tags,under_construction,underground,p_min_pu,...,p_nom_opt,charger_ratio,type,build_year,p_nom_min,p_set,p_max_pu,terrain_factor,ramp_limit_up,ramp_limit_down
Link,,,,,,,,,,,,,,,,,,,,,
AL0 0 urban central gas CHP,EU gas,AL0 0,urban central gas CHP,,0.0,0.0,,NaN,NaN,0.0,...,104.366725,NaN,,0,0.0,0.0,1.0,1.0,NaN,NaN
AT0 0 urban central gas CHP,EU gas,AT0 0,urban central gas CHP,,0.0,0.0,,NaN,NaN,0.0,...,2206.113812,NaN,,0,0.0,0.0,1.0,1.0,NaN,NaN
AT0 1 urban central gas CHP,EU gas,AT0 1,urban central gas CHP,,0.0,0.0,,NaN,NaN,0.0,...,186.336666,NaN,,0,0.0,0.0,1.0,1.0,NaN,NaN
AT0 2 urban central gas CHP,EU gas,AT0 2,urban central gas CHP,,0.0,0.0,,NaN,NaN,0.0,...,151.861355,NaN,,0,0.0,0.0,1.0,1.0,NaN,NaN
BA0 0 urban central gas CHP,EU gas,BA0 0,urban central gas CHP,,0.0,0.0,,NaN,NaN,0.0,...,235.885958,NaN,,0,0.0,0.0,1.0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SE3 5 urban central gas CHP,EU gas,SE3 5,urban central gas CHP,,0.0,0.0,,NaN,NaN,0.0,...,0.238039,NaN,,0,0.0,0.0,1.0,1.0,NaN,NaN
SE3 6 urban central gas CHP,EU gas,SE3 6,urban central gas CHP,,0.0,0.0,,NaN,NaN,0.0,...,1948.478570,NaN,,0,0.0,0.0,1.0,1.0,NaN,NaN
SE3 7 urban central gas CHP,EU gas,SE3 7,urban central gas CHP,,0.0,0.0,,NaN,NaN,0.0,...,0.140468,NaN,,0,0.0,0.0,1.0,1.0,NaN,NaN


In [23]:
stst.links[stst.links.carrier == "urban central solid biomass CHP CC"]

,bus0,bus1,carrier,geometry,length,p_nom,tags,under_construction,underground,p_min_pu,...,p_nom_opt,charger_ratio,type,build_year,p_nom_min,p_set,p_max_pu,terrain_factor,ramp_limit_up,ramp_limit_down
Link,,,,,,,,,,,,,,,,,,,,,
AL0 0 urban central solid biomass CHP CC,EU solid biomass,AL0 0,urban central solid biomass CHP CC,,0.0,0.0,,NaN,NaN,0.0,...,96.324958,NaN,,0,0.0,0.0,1.0,1.0,NaN,NaN
AT0 0 urban central solid biomass CHP CC,EU solid biomass,AT0 0,urban central solid biomass CHP CC,,0.0,0.0,,NaN,NaN,0.0,...,3553.439874,NaN,,0,0.0,0.0,1.0,1.0,NaN,NaN
AT0 1 urban central solid biomass CHP CC,EU solid biomass,AT0 1,urban central solid biomass CHP CC,,0.0,0.0,,NaN,NaN,0.0,...,190.422232,NaN,,0,0.0,0.0,1.0,1.0,NaN,NaN
AT0 2 urban central solid biomass CHP CC,EU solid biomass,AT0 2,urban central solid biomass CHP CC,,0.0,0.0,,NaN,NaN,0.0,...,1330.787761,NaN,,0,0.0,0.0,1.0,1.0,NaN,NaN
BA0 0 urban central solid biomass CHP CC,EU solid biomass,BA0 0,urban central solid biomass CHP CC,,0.0,0.0,,NaN,NaN,0.0,...,308.111935,NaN,,0,0.0,0.0,1.0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SE3 5 urban central solid biomass CHP CC,EU solid biomass,SE3 5,urban central solid biomass CHP CC,,0.0,0.0,,NaN,NaN,0.0,...,0.024738,NaN,,0,0.0,0.0,1.0,1.0,NaN,NaN
SE3 6 urban central solid biomass CHP CC,EU solid biomass,SE3 6,urban central solid biomass CHP CC,,0.0,0.0,,NaN,NaN,0.0,...,0.036928,NaN,,0,0.0,0.0,1.0,1.0,NaN,NaN
SE3 7 urban central solid biomass CHP CC,EU solid biomass,SE3 7,urban central solid biomass CHP CC,,0.0,0.0,,NaN,NaN,0.0,...,0.021204,NaN,,0,0.0,0.0,1.0,1.0,NaN,NaN
